In [67]:
import time
import glob
import numpy as np
import pandas as pd
import os
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
from keras import models
from keras.engine import Model
from keras.layers import Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
np.random.seed(42)
tf.random.set_seed(42)

In [68]:
"""
    Thanks to the suggestion in the following link for getting CUDA to work on Ampere:
    https://stackoverflow.com/questions/38009682/how-to-tell-if-tensorflow-is-using-gpu-acceleration-from-inside-python-shell

    Thanks also to this enlightening article which has a detailed workthrough of a transfer
    learning example. Some of the code in this notebook is based off of the code in this article:
    https://towardsdatascience.com/cnn-transfer-learning-fine-tuning-9f3e7c5806b2

"""
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [69]:
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
#config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [70]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

True

In [71]:
NUM_CLASSES = 43
WIDTH = 48
HEIGHT = 48
CHANNELS = 3

In [72]:
def load_transform_train_data(folder):
    start = time.time()
    image_data = []
    image_labels = []
    #NUM_CLASSES = 43
    #width = 48
    #height = 48
    #channels = 3

    input_path = 'data/archive/'

    for i in range(NUM_CLASSES):
        path = input_path + '{}/'.format(folder) + str(i)
        print(path)
        images = os.listdir(path)

        for img in images:
            try:
                image = cv2.imread(path + '/' + img)
                image_fromarray = Image.fromarray(image, 'RGB')
                resize_image = image_fromarray.resize((HEIGHT, WIDTH))
                image_data.append(np.array(resize_image))
                image_labels.append(i)
            except:
                print('Error - Image loading')

    image_data = np.array(image_data)
    image_labels = np.array(image_labels)
    end = time.time()
    print('Took {} seconds'.format(round(end-start, 5)))
    if folder == 'Train':
        print('Loaded Training data, splitting and normalising...')
        shuffle_indexes = np.arange(image_data.shape[0])
        np.random.shuffle(shuffle_indexes)
        image_data = image_data[shuffle_indexes]
        image_labels = image_labels[shuffle_indexes]
        X_train, X_valid, y_train, y_valid = train_test_split(image_data, image_labels, test_size=0.2, random_state=42, shuffle=True)

        X_train = X_train/255
        X_valid = X_valid/255

        print('X_train.shape {}'.format(X_train.shape))
        print('X_valid.shape {}'.format(X_valid.shape))
        
        y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
        y_valid = keras.utils.to_categorical(y_valid, NUM_CLASSES)

        print('y_train.shape {}'.format(y_train.shape))
        print('y_valid.shape {}'.format(y_valid.shape))
        
        return X_train, X_valid, y_train, y_valid

    elif folder == 'Test':
        print('Loaded Testing data, normalising...')
        X_test = image_data/255
        
        print('X_test.shape {}'.format(X_test.shape))
        
        y_test = keras.utils.to_categorical(image_labels, NUM_CLASSES)
        
        print('y_test.shape {}'.format(y_test.shape))
        
        return X_test, y_test
    
    return image_data, image_labels

In [73]:
start = time.time()

test_path = 'data/archive/Test\\'
xpaths = glob.glob('{}*.png'.format(test_path))
test_df = pd.read_csv('data/archive/Test.csv')

image_data = []
image_labels = []

for path in xpaths:
    img_name = path.replace(test_path, 'Test/')
    #print(img_name)
    img_class = test_df[test_df['Path'] == img_name]['ClassId'].values[0]
    image_labels.append(img_class)
    try:
        image = cv2.imread(path)
        image_fromarray = Image.fromarray(image, 'RGB')
        resize_image = image_fromarray.resize((HEIGHT, WIDTH))
        image_data.append(np.array(resize_image))
        #image_labels.append(i)
    except:
        print('Error - Image loading')
        
image_data = np.array(image_data)
image_labels = np.array(image_labels)
end = time.time()
print('Took {} seconds'.format(round(end-start, 5)))

X_test = image_data/255

print('X_test.shape {}'.format(X_test.shape))

y_test = keras.utils.to_categorical(image_labels, NUM_CLASSES)

print('y_test.shape {}'.format(y_test.shape))

Took 21.79365 seconds
X_test.shape (12630, 48, 48, 3)
y_test.shape (12630, 43)


In [74]:
X_train, X_valid, y_train, y_valid = load_transform_train_data('Train')

data/archive/Train/0
data/archive/Train/1
data/archive/Train/2
data/archive/Train/3
data/archive/Train/4
data/archive/Train/5
data/archive/Train/6
data/archive/Train/7
data/archive/Train/8
data/archive/Train/9
data/archive/Train/10
data/archive/Train/11
data/archive/Train/12
data/archive/Train/13
data/archive/Train/14
data/archive/Train/15
data/archive/Train/16
data/archive/Train/17
data/archive/Train/18
data/archive/Train/19
data/archive/Train/20
data/archive/Train/21
data/archive/Train/22
data/archive/Train/23
data/archive/Train/24
data/archive/Train/25
data/archive/Train/26
data/archive/Train/27
data/archive/Train/28
data/archive/Train/29
data/archive/Train/30
data/archive/Train/31
data/archive/Train/32
data/archive/Train/33
data/archive/Train/34
data/archive/Train/35
data/archive/Train/36
data/archive/Train/37
data/archive/Train/38
data/archive/Train/39
data/archive/Train/40
data/archive/Train/41
data/archive/Train/42
Took 26.32974 seconds
Loaded Training data, splitting and normal

In [75]:
keras.backend.clear_session()
np.random.seed(1)
tf.random.set_seed(1)

In [76]:
input_shape = (WIDTH, HEIGHT, CHANNELS)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0     

In [77]:
# Freeze every layer in base model so they don't train
for layer in base_model.layers:
    layer.trainable = False
    print('Layer {} frozen'.format(layer.name))

Layer input_1 frozen
Layer block1_conv1 frozen
Layer block1_conv2 frozen
Layer block1_pool frozen
Layer block2_conv1 frozen
Layer block2_conv2 frozen
Layer block2_pool frozen
Layer block3_conv1 frozen
Layer block3_conv2 frozen
Layer block3_conv3 frozen
Layer block3_pool frozen
Layer block4_conv1 frozen
Layer block4_conv2 frozen
Layer block4_conv3 frozen
Layer block4_pool frozen
Layer block5_conv1 frozen
Layer block5_conv2 frozen
Layer block5_conv3 frozen
Layer block5_pool frozen


In [78]:
# Take the last layer of model and add to classifier
last = base_model.layers[-1].output
x = Flatten()(last)
x = Dense(1000, activation='relu', name='fc1')(x)
x = Dropout(0.3)(x)
x = Dense(NUM_CLASSES, activation='softmax', name='predictions')(x)
model = Model(base_model.input, x)

In [79]:
model.compile(optimizer=Adam(lr=0.001),
             loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0     

In [81]:
start = time.time()
epochs = 20
batch_size = 256

model.fit(X_train, y_train, batch_size=batch_size,
         validation_data=(X_valid, y_valid),
         epochs=epochs)
end = time.time()
print('Training took {} seconds'.format(round(end-start, 5)))

Epoch 1/20
123/123 [==============================] - 5s 40ms/step - loss: 0.2291 - accuracy: 0.9332 - val_loss: 0.3078 - val_accuracy: 0.9149
Epoch 2/20
123/123 [==============================] - 5s 38ms/step - loss: 0.2170 - accuracy: 0.9376 - val_loss: 0.3060 - val_accuracy: 0.9121
Epoch 3/20
123/123 [==============================] - 5s 38ms/step - loss: 0.2071 - accuracy: 0.9398 - val_loss: 0.2977 - val_accuracy: 0.9174
Epoch 4/20
123/123 [==============================] - 5s 39ms/step - loss: 0.1952 - accuracy: 0.9442 - val_loss: 0.2919 - val_accuracy: 0.9172
Epoch 5/20
123/123 [==============================] - 5s 38ms/step - loss: 0.1860 - accuracy: 0.9456 - val_loss: 0.2921 - val_accuracy: 0.9139
Epoch 6/20
123/123 [==============================] - 5s 38ms/step - loss: 0.1753 - accuracy: 0.9506 - val_loss: 0.2749 - val_accuracy: 0.9241
Epoch 7/20
123/123 [==============================] - 5s 39ms/step - loss: 0.1693 - accuracy: 0.9508 - val_loss: 0.2746 - val_accuracy: 0.9194

In [82]:
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(scores[0]))
print('Test Accuracy: {}'.format(scores[1]))

395/395 [==============================] - 4s 9ms/step - loss: 2.0551 - accuracy: 0.6189: 0s - loss: 2.0551 - accuracy: 0.61
Test Loss: 2.0550825595855713
Test Accuracy: 0.6189231872558594


# Retrain Some Layers

In [83]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0     

In [84]:
for layer in base_model.layers:
    if layer.name == 'block5_conv1':
        break
    layer.trainable = False
    print('Layer {} frozen.'.format(layer.name))

Layer input_2 frozen.
Layer block1_conv1 frozen.
Layer block1_conv2 frozen.
Layer block1_pool frozen.
Layer block2_conv1 frozen.
Layer block2_conv2 frozen.
Layer block2_pool frozen.
Layer block3_conv1 frozen.
Layer block3_conv2 frozen.
Layer block3_conv3 frozen.
Layer block3_pool frozen.
Layer block4_conv1 frozen.
Layer block4_conv2 frozen.
Layer block4_conv3 frozen.
Layer block4_pool frozen.


In [85]:
# Take the last layer of model and add to classifier
last = base_model.layers[-1].output
x = Flatten()(last)
x = Dense(1000, activation='relu', name='fc1')(x)
x = Dropout(0.3)(x)
x = Dense(NUM_CLASSES, activation='softmax', name='predictions')(x)
model = Model(base_model.input, x)

In [86]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [87]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0   

In [88]:
start = time.time()
epochs = 20
batch_size = 256

model.fit(X_train, y_train, batch_size=batch_size,
         validation_data=(X_valid, y_valid),
         epochs=epochs)
end = time.time()
print('Training took {} seconds'.format(round(end-start, 5)))

Epoch 1/20
123/123 [==============================] - 7s 52ms/step - loss: 2.5643 - accuracy: 0.2599 - val_loss: 0.9395 - val_accuracy: 0.6765
Epoch 2/20
123/123 [==============================] - 6s 51ms/step - loss: 0.7904 - accuracy: 0.7238 - val_loss: 0.3650 - val_accuracy: 0.8804
Epoch 3/20
123/123 [==============================] - 6s 49ms/step - loss: 0.3254 - accuracy: 0.8909 - val_loss: 0.2588 - val_accuracy: 0.9153
Epoch 4/20
123/123 [==============================] - 6s 45ms/step - loss: 0.1665 - accuracy: 0.9453 - val_loss: 0.1662 - val_accuracy: 0.9481
Epoch 5/20
123/123 [==============================] - 6s 45ms/step - loss: 0.1203 - accuracy: 0.9606 - val_loss: 0.1651 - val_accuracy: 0.9509
Epoch 6/20
123/123 [==============================] - 6s 45ms/step - loss: 0.0828 - accuracy: 0.9725 - val_loss: 0.0991 - val_accuracy: 0.9725
Epoch 7/20
123/123 [==============================] - 6s 45ms/step - loss: 0.0653 - accuracy: 0.9774 - val_loss: 0.1545 - val_accuracy: 0.9509

In [89]:
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(scores[0]))
print('Test Accuracy: {}'.format(scores[1]))

395/395 [==============================] - 3s 8ms/step - loss: 1.2842 - accuracy: 0.8084
Test Loss: 1.2841975688934326
Test Accuracy: 0.8083927035331726


# Retrain More Layers

In [90]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0     

In [91]:
for layer in base_model.layers:
    if layer.name == 'block4_conv1':
        break
    layer.trainable = False
    print('Layer {} frozen.'.format(layer.name))

Layer input_3 frozen.
Layer block1_conv1 frozen.
Layer block1_conv2 frozen.
Layer block1_pool frozen.
Layer block2_conv1 frozen.
Layer block2_conv2 frozen.
Layer block2_pool frozen.
Layer block3_conv1 frozen.
Layer block3_conv2 frozen.
Layer block3_conv3 frozen.
Layer block3_pool frozen.


In [92]:
# Take the last layer of model and add to classifier
last = base_model.layers[-1].output
x = Flatten()(last)
x = Dense(1000, activation='relu', name='fc1')(x)
x = Dropout(0.3)(x)
x = Dense(NUM_CLASSES, activation='softmax', name='predictions')(x)
model = Model(base_model.input, x)

In [93]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [94]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0   

In [95]:
start = time.time()
epochs = 20
batch_size = 256

model.fit(X_train, y_train, batch_size=batch_size,
         validation_data=(X_valid, y_valid),
         epochs=epochs)
end = time.time()
print('Training took {} seconds'.format(round(end-start, 5)))

Epoch 1/20
123/123 [==============================] - 10s 64ms/step - loss: 3.1392 - accuracy: 0.1425 - val_loss: 1.4234 - val_accuracy: 0.4857
Epoch 2/20
123/123 [==============================] - 8s 61ms/step - loss: 1.1074 - accuracy: 0.6229 - val_loss: 0.5758 - val_accuracy: 0.7919
Epoch 3/20
123/123 [==============================] - 8s 62ms/step - loss: 0.4048 - accuracy: 0.8587 - val_loss: 0.2198 - val_accuracy: 0.9285
Epoch 4/20
123/123 [==============================] - 8s 62ms/step - loss: 0.1988 - accuracy: 0.9366 - val_loss: 0.2311 - val_accuracy: 0.9440
Epoch 5/20
123/123 [==============================] - 8s 61ms/step - loss: 0.1415 - accuracy: 0.9563 - val_loss: 0.0850 - val_accuracy: 0.9732
Epoch 6/20
123/123 [==============================] - 8s 61ms/step - loss: 0.0789 - accuracy: 0.9752 - val_loss: 0.0842 - val_accuracy: 0.9767
Epoch 7/20
123/123 [==============================] - 8s 62ms/step - loss: 0.0783 - accuracy: 0.9778 - val_loss: 0.1204 - val_accuracy: 0.970

In [96]:
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(scores[0]))
print('Test Accuracy: {}'.format(scores[1]))

395/395 [==============================] - 3s 8ms/step - loss: 0.3379 - accuracy: 0.9355
Test Loss: 0.33793559670448303
Test Accuracy: 0.9354711174964905


# Train full model

In [97]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0     

In [98]:
# Take the last layer of model and add to classifier
last = base_model.layers[-1].output
x = Flatten()(last)
x = Dense(1000, activation='relu', name='fc1')(x)
x = Dropout(0.3)(x)
x = Dense(NUM_CLASSES, activation='softmax', name='predictions')(x)
model = Model(base_model.input, x)

In [99]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [100]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0   

In [101]:
start = time.time()
epochs = 20
batch_size = 256

model.fit(X_train, y_train, batch_size=batch_size,
         validation_data=(X_valid, y_valid),
         epochs=epochs)
end = time.time()
print('Training took {} seconds'.format(round(end-start, 5)))

Epoch 1/20
123/123 [==============================] - 13s 103ms/step - loss: 3.5326 - accuracy: 0.0688 - val_loss: 2.9138 - val_accuracy: 0.1417
Epoch 2/20
123/123 [==============================] - 12s 100ms/step - loss: 2.6573 - accuracy: 0.1945 - val_loss: 1.1304 - val_accuracy: 0.5935
Epoch 3/20
123/123 [==============================] - 12s 100ms/step - loss: 0.8714 - accuracy: 0.6782 - val_loss: 0.1882 - val_accuracy: 0.9410
Epoch 4/20
123/123 [==============================] - 12s 101ms/step - loss: 0.1673 - accuracy: 0.9480 - val_loss: 0.0848 - val_accuracy: 0.9769
Epoch 5/20
123/123 [==============================] - 12s 100ms/step - loss: 0.0587 - accuracy: 0.9851 - val_loss: 0.0435 - val_accuracy: 0.9889
Epoch 6/20
123/123 [==============================] - 12s 101ms/step - loss: 0.0327 - accuracy: 0.9914 - val_loss: 0.0855 - val_accuracy: 0.9784
Epoch 7/20
123/123 [==============================] - 12s 100ms/step - loss: 0.0639 - accuracy: 0.9819 - val_loss: 0.0465 - val_ac

In [48]:
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(scores[0]))
print('Test Accuracy: {}'.format(scores[1]))

395/395 [==============================] - 3s 8ms/step - loss: 0.1597 - accuracy: 0.9683
Test Loss: 0.15971246361732483
Test Accuracy: 0.9683293700218201


In [51]:
X_train[0].shape

(48, 48, 3)